In [1]:
# import all relevant libraries
import pandas as pd
import numpy as np
import uuid
import re
import json
from pandas.plotting import register_matplotlib_converters
from pandas import json_normalize

In [2]:
# import weather data
weather = pd.read_csv("weather_burbank_airport.csv", encoding = "ISO-8859-1")
weather.head()

,city,timestamp,temperature,cloud_cover,cloud_cover_description,pressure,windspeed,precipitation,felt_temperature
0,Burbank,2018-01-01 08:53:00,9.0,33.0,Fair,991.75,9.0,0.0,8.0
1,Burbank,2018-01-01 09:53:00,9.0,33.0,Fair,992.08,0.0,0.0,9.0
2,Burbank,2018-01-01 10:53:00,9.0,21.0,Haze,992.08,0.0,0.0,9.0
3,Burbank,2018-01-01 11:53:00,9.0,29.0,Partly Cloudy,992.08,0.0,0.0,9.0
4,Burbank,2018-01-01 12:53:00,8.0,33.0,Fair,992.08,0.0,0.0,8.0


In [3]:
print(weather['city'].unique())

['Burbank']


As we can see, the city column has only one value: Burbank, which is a city northwest to Los Angeles. We can therefore remove this column as this does not help us.

In [4]:
weather = weather.drop(columns=['city'])
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29244 entries, 0 to 29243
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   timestamp                29244 non-null  object 
 1   temperature              29219 non-null  float64
 2   cloud_cover              29224 non-null  float64
 3   cloud_cover_description  29224 non-null  object 
 4   pressure                 29236 non-null  float64
 5   windspeed                29158 non-null  float64
 6   precipitation            29244 non-null  float64
 7   felt_temperature         29218 non-null  float64
dtypes: float64(6), object(2)
memory usage: 1.8+ MB


In [7]:
#correct datetime
weather['timestamp'] = pd.to_datetime(weather['timestamp'])
print(weather.dtypes)



timestamp                  datetime64[ns]
temperature                       float64
cloud_cover                       float64
cloud_cover_description            object
pressure                          float64
windspeed                         float64
precipitation                     float64
felt_temperature                  float64
dtype: object


In [8]:
# print for each row how many null values there are
print(weather.isna().sum())

timestamp                   0
temperature                25
cloud_cover                20
cloud_cover_description    20
pressure                    8
windspeed                  86
precipitation               0
felt_temperature           26
dtype: int64


We have 22 different types of description (one category is NaN)